# PERP-UNI NVP Spread Investment Monitor

In [144]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import data
import analytics

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Inputs

In [145]:
start_date = '2020-09-09'
end_date = '2021-09-24'
dexs = [
    'uniswap_v1',
    'uniswap_v2',
    'uniswap',
    'perpetual_protocol',
]
coins = [
    'uniswap',
    'perpetual-protocol',
]
benchmark = 'defipulse-index'
nvf_avg_window = 30
annualization_factor = 365
uni_trade_fee = 0.003
perp_trade_fee = 0.001

## Get Data

In [149]:
# Call CoinGecko for market data
coins.append(benchmark)
dex_trading_volume = data.dex_trading_volume(dexs, start_date, end_date)
coin_market_data = data.coin_market_data(coins, start_date, end_date)

# Prepare DEX trading volume data
dex_trading_volume = dex_trading_volume.fillna(0)
uni_tv = dex_trading_volume['uniswap'] + dex_trading_volume['uniswap_v2'] + dex_trading_volume['uniswap_v1']
perp_tv = dex_trading_volume['perpetual_protocol']

# Prepare coin and benchmark market data
uni_price = coin_market_data['uniswap price']
uni_mc = coin_market_data['uniswap market cap']
perp_price = coin_market_data['perpetual-protocol price']
perp_mc = coin_market_data['perpetual-protocol market cap']
benchmark_price = coin_market_data['defipulse-index price']
benchmark_mc = coin_market_data['defipulse-index market cap']

# Import and format trade data
funding_payments = pd.read_csv('input/funding_payments.csv', index_col='time', parse_dates=True)
pnl = pd.read_csv('input/pnl.csv', index_col='day')
pnl = pnl.drop(' Total')
pnl.index = pnl.index.map(lambda x: pd.to_datetime(float(x), unit='s'))

# Prepare trade data
funding_payments = funding_payments.groupby('future').resample('D').sum()
perp_fp = funding_payments.loc['PERP-PERP']['payment']
perp_fp.index = perp_fp.index.tz_localize(None)
uni_fp = funding_payments.loc['UNI-PERP']['payment']
uni_fp.index = uni_fp.index.tz_localize(None)
perp_pnl = pnl['PERP-PERP'].sort_index()
uni_pnl = pnl['UNI-PERP'].sort_index()

Getting DEX trading volume...
Getting coin market data...
- bitcoin
Got coin market data
- uniswap_v1
- uniswap_v2
- perpetual_protocol
- uniswap
Got DEX trading volume
Getting coin market data...
- uniswap
- perpetual-protocol
- defipulse-index
Got coin market data


## Compute Analytics

In [152]:
# NVF
uni_nvf = analytics.nvf(uni_mc, uni_tv, uni_trade_fee, nvf_avg_window, annualization_factor)
perp_nvf = analytics.nvf(perp_mc, perp_tv, perp_trade_fee, nvf_avg_window, annualization_factor)

# NVF spread (absolute & % diff)
abs_nvf_spread = (perp_nvf - uni_nvf).dropna()
perc_nvf_spread = (abs_nvf_spread / uni_nvf).dropna()

# TODO: Overview trade stats


## Present Analytics

## Output Analytics